In [1]:
%matplotlib inline

In [2]:
import os
import layoutparser as lp
import cv2
import pdf2image
import zipfile

from matplotlib.pyplot import imshow
import numpy as np

In [1]:
path2papers = '/Users/hgt6rn/Desktop/20231019131106578.pdf'

images = pdf2image.convert_from_path(path2papers)
images = [np.array(img) for img in images]
#print(images)

imshow(np.asarray(images[0]))

NameError: name 'pdf2image' is not defined

In [4]:
# mask_rcnn_X_101_32x8d_FPN_3x

# faster_rcnn_R_50_FPN_3x


model = lp.models.Detectron2LayoutModel(
            #config_path ='lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config', # In model catalog  #
            config_path = '/Users/hgt6rn/Documents/DataSets/layoutparser/faster_rcnn_R_50_FPN_3x_config.yml',
            model_path =  '/Users/hgt6rn/Documents/DataSets/layoutparser/faster_rcnn_R_50_FPN_3x_model.pth',
            label_map   ={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}, # In model`label_map`
            extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # Optional
        )

In [5]:
layout = model.detect(images[0])
lp.draw_box(images[0], layout, box_width=3)

/opt/homebrew/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
for b in layout:
    print(b.type)

text_blocks = lp.Layout([b for b in layout if b.type=='Table'])
figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])
print(len(text_blocks))
#text_blocks = lp.Layout([b for b in text_blocks \
#                   if not any(b.is_in(b_fig) for b_fig in figure_blocks)])

Title
Text
Title
Table
Title
Table
2


In [13]:
ocr_agent = lp.TesseractAgent()
# Initialize the tesseract ocr engine. You might need 
# to install the OCR components in layoutparser:
# pip install layoutparser[ocr]


for block in text_blocks:
    segment_image = (block
                       .pad(left=5, right=5, top=5, bottom=5)
                       .crop_image(images[0]))
        # add padding in each image segment can help
        # improve robustness 
        
    text = ocr_agent.detect(segment_image)
    block.set(text=text, inplace=True)

In [14]:
for txt in text_blocks.get_texts():
    print(txt, end='\n---\n')

University oT Virginia (UVa]

Charlottesville, VA 22903
fel: (434) 924-0311 Internet: www.virginia.edu

frrollment: 20,000 Faculty: 2,000 Endowment: Over $2.5 billion
‘Year Founded: 1819

Administration
president Dr. Teresa A. Sullivan - (434) 924-3337
‘E-mail: tas6n@virginia.edu ‘ax: (434) 924-3792
_ Education: Michigan State 1970 BA; Chicago 1972 AM, 1975 PhD
Chief of Staff and Associate Vice President,

‘Administration Nancy A. Rivers ... -.. - (434) 982-2662
Chancellor, College at Wise Dr. Donna Price-Henry ... .. (276) 328-0122
{College Ave., Wise, VA 24293
mail: dph3p@virginia.edu
ducation: Pennsylvania 1982 BA; Thomas Jefferson PhD
Executive Vice President and Chief Operating Officer .
atrick D. Hogan... 2.00... ce es eee cece ee (434).924-3252
xecutive Vice President and Provost
john D. Simon, PhD... .... 22.0... (434) 982-2362
mail: jds2ts@virginia.edu :

Education: Williams 1979 BA; Harvard 1981, 1983 PhD

wost and Senior Vice Chancellor, College at Wise

hin'Sanders Huguenin

In [19]:
text = ocr_agent.detect(images[0])
print(text)

University of Virginia 685

University of Virginia [UVa]

. Trust
meee Charlottesville, VA 22903
fel: (434) 924-0311 Internet: www.virginia.edu
frrollment: 20,000 Faculty: 2,000 Endowment: Over $2.5 billion
hairman ‘Year Founded: 1819
Trustee

Administration
resident Dr. Teresa A. Sullivan. .................... (434) 924-3337
E-mail: tas6n@virginia.edu Fax: (434) 924-3792

. Trustee _ Education: Michigan State 1970 BA; Chicago 1972 AM, 1975 PhD
Chief of Staff and Associate Vice President,
Administration Nancy A. Rivers.................. (434) 982-2662
Chancellor, College at Wise Dr. Donna Price-Henry ... .. (276) 328-0122

{College Ave., Wise, VA 24293
E-mail: dph3p@virginia.edu
Education: Pennsylvania 1982 BA; Thomas Jefferson PhD
ecutive Vice President and Chief Operating Officer
Patrick D. Hogan .
xecutive Vice President and Provost
John D. Simon, PhD.......0 00.000... 00. .00 cee (434) 982-2362
Femail: jds2ts@virginia.edu :
_ Education: Williams 1979 BA; Harvard 1981, 1983 PhD
rovo

In [ ]:
Name | Title | School | Year